# Competing phase generator
Read in the compounds of interest and come up with a set of likely competing phases. 

1. The Materials Project database is used to estimate which points on the phase diagram need to be explicitly calculated for each compound.

2. This list of competing phases is reduced to a minimum set and saved in a json file.

Note: A similar workflow can be used after DFT calculations to construct phase diagrams with newly-calculated energies and get e_hull values.

In [16]:
import json
from pymatgen import MPRester
from pymatgen import Structure, Specie, Element, Composition
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDEntry
import os
from tqdm import tqdm

from functions.competing_phases import get_competing, get_decomp_product_ids

%load_ext autoreload
%autoreload 2

# Put your own API key in here (or set it as an environment variable like below)
mpr = MPRester(os.environ.get('MP_API_KEY'))

The input must be a list of dicts in which the key 'structure' is a pymatgen structure. We load these in and get them into a useful format below:

In [2]:
# Load in the structures
with open('data/example_input_structures.json', 'r') as f:
    structures = json.load(f)

# Convert back to pymatgen structure objects
for i in structures:
    i['structure'] = Structure.from_dict(i['structure'])
    
print("{} structures imported".format(len(structures)))

# Add "chemsys" key to dicts (unique elements in structure)
for entry in structures:
    entry['chemsys'] = set(entry['structure'].composition.element_composition.elements)

100 structures imported


We download stable structures from the MP database:

In [3]:
# Download formulas and e_hull for all stable compounds in MP  (takes a couple of mins)
criteria = {'e_above_hull': 0}
properties = ['full_formula', 'task_id', 'final_energy']
mp_db = mpr.query(criteria,properties)

# Add "chemsys" key to mp_db
for entry in mp_db:
    entry['chemsys'] = set(Composition(entry['full_formula']).elements)

We find competing phases for each structure by searching mp_db for entries with a relevant "chemsys" and creating phase diagram objects:

In [4]:
competing_phases_sets = []
for struc in tqdm(structures):
    competing_phases_sets.append(get_competing(struc, mp_db))

100%|██████████| 100/100 [00:00<00:00, 290.45it/s]


In [21]:
all_decomp_prod_ids = get_decomp_product_ids(structures, competing_phases_sets)

76 unique competing phases to calculate


In [23]:
# Download the actual structures of the competing phases we are interested in and save
criteria = {'task_id': {'$in': all_decomp_prod_ids}}
properties = ['structure', 'task_id']
competing_phase_structures = mpr.query(criteria, properties)

# Save to a json file
for struc in competing_phase_structures:
    struc['structure'] = struc['structure'].as_dict()

with open('data/competing_phases.json', 'w') as f:
    json.dump(competing_phase_structures,f)